In [14]:
t = 6
removed = []
for i in range(1018):
    removed.append(t)
    t += 1
print(removed)

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 22

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import sys
import re

ROOT_URL = 'http://accent.gmu.edu/'
BROWSE_LANGUAGE_URL = 'browse_language.php?function=find&language={}'
WAIT = 1.2
DEBUG = True
''''''
#original function
def get_htmls(urls):
    
    #Retrieves html in text form from ROOT_URL
    #:param urls (list): List of urls from which to retrieve html
    #:return (list): list of HTML strings
    htmls = []
    for url in urls:
        if DEBUG:
            print('downloading from {}'.format(url))
        htmls.append(requests.get(url).text)
        time.sleep(WAIT)
    return(htmls)
'''
def get_htmls(urls):
    
    #Retrieves html in text form from ROOT_URL
    #:param urls (list): List of urls from which to retrieve html
    #:return (list): list of HTML strings
    
    htmls = []
    for url in urls:
        for i in range(3):
            if DEBUG:
                print('downloading from {}'.format(url))
            htmls.append(requests.get(url).text)
            time.sleep(WAIT)
    return(htmls)
'''
def build_search_urls(languages):
    '''
    creates url from ROOT_URL and languages
    :param languages (list): List of languages
    :return (list): List of urls
    '''
    return([ROOT_URL+BROWSE_LANGUAGE_URL.format(language) for language in languages])

def parse_p(p_tag):
    '''
    Extracts href property from HTML <p> tag string
    :param p_tag (str): HTML string
    :return (str): string of link
    '''
    text = p_tag.text.replace(' ','').split(',')
    return([ROOT_URL+p_tag.a['href'], text[0], text[1]])

def get_bio(hrefs):
    '''
    Retrieves HTML from list of hrefs and returns bio information
    :param hrefs (list): list of hrefs
    :return (DataFrame): Pandas DataFrame with bio information
    '''
    htmls = get_htmls(hrefs)
    bss = [BeautifulSoup(html,'html.parser') for html in htmls]
    rows = []
    bio_row = []
    for bs in bss:
        rows.append([li.text for li in bs.find('ul','bio').find_all('li')])
    for row in rows:
        bio_row.append(parse_bio(row))

    return(pd.DataFrame(bio_row))

def parse_bio(row):
    '''
    Parse bio data from row string
    :param row (str): Unparsed bio string
    :return (list): Bio columns
    '''
    cols = []
    for col in row:
        try:
            tmp_col = re.search((r"\:(.+)",col.replace(' ','')).group(1))
        except:
            tmp_col = col
        cols.append(tmp_col)
    return(cols)


def create_dataframe(languages):
    '''
    :param languages (str): language from which you want to get html
    :return df (DataFrame): DataFrame that contains all audio metadata from searched language
    '''
    print("dataframes")
    htmls = get_htmls(build_search_urls(languages))
    bss = [BeautifulSoup(html,'html.parser') for html in htmls]
    persons = []

    for bs in bss:
        for p in bs.find_all('p'):
            if p.a:
                persons.append(parse_p(p))
    df = pd.DataFrame(persons, columns=['href','language_num','sex'])
    #df.drop(labels = removed,axis = 0, inplace = True)
    print(df)
    bio_rows = get_bio(df['href'])

    if DEBUG:
        print('loading finished')

    df['birth_place'] = bio_rows.iloc[:,0]
    df['native_language'] = bio_rows.iloc[:,1]
    df['other_languages'] = bio_rows.iloc[:,2]
    df['age_sex'] = bio_rows.iloc[:,3]
    df['age_of_english_onset'] = bio_rows.iloc[:,4]
    df['english_learning_method'] = bio_rows.iloc[:,5]
    df['english_residence'] = bio_rows.iloc[:,6]
    df['length_of_english_residence'] = bio_rows.iloc[:,7]

    df['birth_place'] = df['birth_place'].apply(lambda x: x[:-6].split(' ')[-2:])
    # print(df['birth_place'])
    # df['birth_place'] = lambda x: x[:-6].split(' ')[2:], df['birth_place']
    df['native_language'] = df['native_language'].apply(lambda x: x.split(' ')[2])
    # print(df['native_language'])
    # df['native_language'] = lambda x: x.split(' ')[2], df['native_language']
    df['other_languages'] = df['other_languages'].apply(lambda x: x.split(' ')[2:])
    # print(df['other_languages'])
    # df['other_languages'] = lambda x: x.split(' ')[2:], df['other_languages']
    df['age_sex'], df['age'] = df['age_sex'].apply(lambda x: x.split(' ')[2:]), df['age_sex'].apply(lambda x: x.replace('sex:','').split(',')[1])
    # print(df['age'])
    # df['age_sex'] = lambda x: x.split(' ')[2], df['age_sex']
    # df['age_of_english_onset'] = lambda x: float(x.split(' ')[-1]), df['age_of_english_onset']
    df['age_of_english_onset'] = df['age_of_english_onset'].apply(lambda x: float(x.split(' ')[-1]))
    # print(df['age_of_english_onset'])
    # df['english_learning_method'] = lambda x: x.split(' ')[-1], df['english_learning_method']
    df['english_learning_method'] = df['english_learning_method'].apply(lambda x: x.split(' ')[-1])
    # print(df['english_learning_method'])
    # df['english_residence'] = lambda x: x.split(' ')[2:], df['english_residence']
    df['english_residence'] = df['english_residence'].apply(lambda x: x.split(' ')[2:])
    # print(df['english_residence'])
    # df['length_of_english_residence'] = lambda x: float(x.split(' ')[-2]), df['length_of_english_residence']
    df['length_of_english_residence'] = df['length_of_english_residence'].apply(lambda x: float(x.split(' ')[-2]))
    # print(df['length_of_english_residence'])

    # df['age'] = lambda x: x.replace(' ','').split(',')[0], df['age_sex']

    return(df)

if __name__ == '__main__':
    '''
    console command example:
    python fromwebsite.py bio_metadata.csv mandarin english arabic
    '''

    df = None

    # Set destination file
    destination_file = 'data_afr_azer_dan.csv'

    # If no language arguments, use 'mandarin' as default
    try:
        languages = ['afrikaans', 'azerbaijani', 'danish']
    except:
        languages = ['english']
        pass

    # Check if destination file exists, else create a new one
    try:
        df = pd.read_csv(destination_file)
        df = df.append(create_dataframe(languages=languages),ignore_index=True)

    except:
        df = create_dataframe(languages=languages)


    df.drop_duplicates(subset='language_num',inplace=True)

    df.to_csv(destination_file,index=False)

dataframes
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=afrikaans
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=azerbaijani
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=danish
                                                 href  language_num     sex
0   http://accent.gmu.edu/browse_language.php?func...    afrikaans1  female
1   http://accent.gmu.edu/browse_language.php?func...    afrikaans2    male
2   http://accent.gmu.edu/browse_language.php?func...    afrikaans3    male
3   http://accent.gmu.edu/browse_language.php?func...    afrikaans4    male
4   http://accent.gmu.edu/browse_language.php?func...    afrikaans5    male
5   http://accent.gmu.edu/browse_language.php?func...    afrikaans6    male
6   http://accent.gmu.edu/browse_language.php?func...  azerbaijani1    male
7   http://accent.gmu.edu/browse_language.php?func...  azerbaijani2  female
8   http://accent.gmu.edu/brow

In [19]:
print(df)

                                                 href  language_num     sex  \
0   http://accent.gmu.edu/browse_language.php?func...    afrikaans1  female   
1   http://accent.gmu.edu/browse_language.php?func...    afrikaans2    male   
2   http://accent.gmu.edu/browse_language.php?func...    afrikaans3    male   
3   http://accent.gmu.edu/browse_language.php?func...    afrikaans4    male   
4   http://accent.gmu.edu/browse_language.php?func...    afrikaans5    male   
5   http://accent.gmu.edu/browse_language.php?func...    afrikaans6    male   
6   http://accent.gmu.edu/browse_language.php?func...  azerbaijani1    male   
7   http://accent.gmu.edu/browse_language.php?func...  azerbaijani2  female   
8   http://accent.gmu.edu/browse_language.php?func...  azerbaijani3    male   
9   http://accent.gmu.edu/browse_language.php?func...  azerbaijani4  female   
10  http://accent.gmu.edu/browse_language.php?func...  azerbaijani5    male   
11  http://accent.gmu.edu/browse_language.php?func..

In [8]:
print(urls)

['http://accent.gmu.edu/browse_language.php?function=find&language=mandarin', 'http://accent.gmu.edu/browse_language.php?function=find&language=english', 'http://accent.gmu.edu/browse_language.php?function=find&language=spanish']


In [22]:
htmls = get_htmls(build_search_urls(languages))

1
2
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=mandarin
3
2
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=english
3
2
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=spanish
3
4


In [23]:
bss = [BeautifulSoup(html,'html.parser') for html in htmls]

In [24]:
print(bss)

[<!DOCTYPE html >

<html>
<head>
<title>speech accent archive: browse</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Piwik -->
<script type="text/javascript">
  var _paq = _paq || [];
  _paq.push(["setDomains", ["*.accent.gmu.edu"]]);
  _paq.push(['trackPageView']);
  _paq.push(['enableLinkTracking']);
  (function() {
    var u="//chnmdev.gmu.edu/piwik/";
    _paq.push(['setTrackerUrl', u+'piwik.php']);
    _paq.push(['setSiteId', 2]);
    var d=document, g=d.createElement('script'), s=d.getElementsByTagName('script')[0];
    g.type='text/javascript'; g.async=true; g.defer=true; g.src=u+'piwik.js'; s.parentNode.insertBefore(g,s);
  })();
</script>
<noscript><p><img alt="" src="//chnmdev.gmu.edu/piwik/piwik.php?idsite=2" style="border:0;"/></p></noscript>
<!-- End Piwik Code -->
</head>
<link href="css/saa.css" rel="stylesheet" type="text/css"/>
<body>
<div id="wrapper">
<div id="banner">
<div id="logolink">
<a href="http://accent.gmu.edu/"><img alt="r

In [24]:
import pandas as pd
import urllib.request
import os
import sys
from pydub import AudioSegment

class GetAudio:

    def __init__(self, csv_filepath, destination_folder= 'Final Project/audio/', wait= 1.5, debug=False ):
        '''
        Initializes GetAudio class object
        :param destination_folder (str): Folder where audio files will be saved
        :param wait (float): Length (in seconds) between web requests
        :param debug (bool): Outputs status indicators to console when True
        '''
        self.csv_filepath = csv_filepath
        self.audio_df = pd.read_csv(csv_filepath)
        self.url = 'http://chnm.gmu.edu/accent/soundtracks/{}.mp3'
        self.destination_folder = destination_folder
        self.wait = wait
        self.debug = False

    def check_path(self):
        '''
        Checks if self.distination_folder exists. If not, a folder called self.destination_folder is created
        '''
        if not os.path.exists(self.destination_folder):
            if self.debug:
                print('{} does not exist, creating'.format(self.destination_folder))
            os.makedirs('../' + self.destination_folder)

    def get_audio(self):
        '''
        Retrieves all audio files from 'language_num' column of self.audio_df
        If audio file already exists, move on to the next
        :return (int): Number of audio files downloaded
        '''

        self.check_path()

        counter = 0

        for lang_num in self.audio_df['language_num']:
            if not os.path.exists(self.destination_folder +'{}.wav'.format(lang_num)):
                if self.debug:
                    print('downloading {}'.format(lang_num))
                (filename, headers) = urllib.request.urlretrieve(self.url.format(lang_num))
                sound = AudioSegment.from_mp3(filename)
                sound.export('../' + self.destination_folder + "{}.wav".format(lang_num), format="wav")
                counter += 1

        return counter

if __name__ == '__main__':
    '''
    Example console command
    python GetAudio.py audio_metadata.csv
    '''
    csv_file = 'data_afr_azer_dan.csv'
    ga = GetAudio(csv_filepath=csv_file)
    ga.get_audio()

In [11]:
conda install ffmpeg

Solving environment: done

## Package Plan ##

  environment location: /Users/Nicholas/opt/anaconda3

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.0               |   py38hecd8cb5_0         2.9 MB
    ffmpeg-4.0                 |       h01ea3c9_0        21.8 MB
    libopus-1.3.1              |       h1de35cc_0         480 KB
    libvpx-1.7.0               |       h378b8a2_0         1.3 MB
    ------------------------------------------------------------
                                           Total:        26.4 MB

The following NEW packages will be INSTALLED:

  ffmpeg             pkgs/main/osx-64::ffmpeg-4.0-h01ea3c9_0
  libopus            pkgs/main/osx-64::libopus-1.3.1-h1de35cc_0
  libvpx             pkgs/main/osx-64::libvpx-1.7.0-h378b8a2_0

The following packages will be UPDATED:

  conda                                4.9.2-p